In [2]:
''' Load raw data '''

import pandas as pd
import numpy as np
from pathlib import Path

base_dir = Path.cwd().parent
file_path = base_dir/'data'/'raw_data.csv'

df = pd.read_csv(file_path)
print(df.head())

   patient_id  gender  age  visit_date   department  physician_id  \
0      154669    Male   43  2024-11-05   Cardiology         30927   
1      144776  Female   56  2024-12-15   Cardiology         14816   
2      217503    Male   49  2024-01-02  Orthopedics         73819   
3      174851    Male    8  2024-06-29   Pediatrics         81325   
4      193134  Female   39  2024-11-21   Cardiology         10046   

       diagnosis  visit_type       visit_reason appointment_id is_emergency  \
0   Hypertension  Outpatient                NaN         A00000          Yes   
1   Hypertension  Outpatient         Chest Pain         A00001           No   
2      Back Pain   Inpatient  Surgery Follow-up         A00002           No   
3         Asthma  Outpatient              Fever         A00003           No   
4  Heart Failure  Outpatient         Chest Pain         A00004          Yes   

  insurance_id                    payer_name  payer_type claim_status  \
0   INS-MMIAPD                  Flore

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   patient_id          6000 non-null   int64 
 1   gender              6000 non-null   object
 2   age                 6000 non-null   int64 
 3   visit_date          6000 non-null   object
 4   department          6000 non-null   object
 5   physician_id        6000 non-null   int64 
 6   diagnosis           5576 non-null   object
 7   visit_type          6000 non-null   object
 8   visit_reason        5601 non-null   object
 9   appointment_id      6000 non-null   object
 10  is_emergency        6000 non-null   object
 11  insurance_id        3697 non-null   object
 12  payer_name          5598 non-null   object
 13  payer_type          6000 non-null   object
 14  claim_status        6000 non-null   object
 15  charge_amount_USD   6000 non-null   int64 
 16  payment_amount_USD  6000

In [23]:
''' Get unique values for each column '''
for column in df.columns:
    print(f'Unique values in {column} column: {df[column].unique()}\n')


Unique values in patient_id column: [154669 144776 217503 ... 146624 119838 154688]

Unique values in gender column: ['Male' 'Female']

Unique values in age column: [43 56 49  8 39 12 29 41 13 36 17 50 61 68 73 31 34 30 59 10 40 67 37 22
 57 58 48 47 60 70 72 46 27 54 11  6  4 35 44 15 42 24 16  5 69 62 26 33
 28 38 66 14 80 45 32 65 51 25  3 20 52 23 53  9 21  2 18  7  1 76 63 77
 75 55 71 74 78 64 19 79]

Unique values in visit_date column: ['2024-11-05' '2024-12-15' '2024-01-02' '2024-06-29' '2024-11-21'
 '2024-01-01' '2025-06-20' '2024-03-25' '2025-02-26' '2024-10-07'
 '2025-06-25' '2023-07-20' '2024-06-22' '2024-06-13' '2024-11-17'
 '2024-01-07' '2024-01-23' '2025-02-24' '2023-11-28' '2025-01-30'
 '2025-05-15' '2024-01-10' '2023-10-17' '2024-10-31' '2025-02-09'
 '2024-08-20' '2023-08-28' '2024-09-01' '2025-02-08' '2024-11-11'
 '2025-02-17' '2023-09-10' '2025-01-09' '2024-06-02' '2024-11-03'
 '2025-04-30' '2025-05-12' '2023-11-17' '2024-05-18' '2025-06-04'
 '2024-06-16' '2025-01-11

In [38]:
''' Check / Resolve tally of missing values or nan in specific columns '''

columns_to_check = ['diagnosis', 'visit_reason', 'insurance_id', 'payer_name', 'admission_date', 'discharge_date']
for col in columns_to_check:
    total_rows = 6000

    not_null_count = df[col].notna().sum()
    missing_count = total_rows - not_null_count

    if missing_count != df[col].isna().sum():
        print(f'Discrepancy found in {col} column!')
    else:
        print(f'\n{col} column: {missing_count} missing values confirmed.')



diagnosis column: 424 missing values confirmed.

visit_reason column: 399 missing values confirmed.

insurance_id column: 2303 missing values confirmed.

payer_name column: 402 missing values confirmed.

admission_date column: 2988 missing values confirmed.

discharge_date column: 2988 missing values confirmed.


In [5]:
''' Rename attribute 'insurance_id' to 'payer_id' '''

df = df.rename(columns={'insurance_id': 'payer_id'})

In [11]:
''' Resolve null values in 'insurance_id' '''

df['payer_id'] = df['payer_id'].fillna('INS_NULL')

In [8]:
''' Change dtype of 'is_emergency' from str to boolean (Yes : TRUE, No: FALSE) '''

bool_map = {'Yes': True, 'No': False}
df['is_emergency'] = df['is_emergency'].map(bool_map)

print(df['is_emergency'].head())

0     True
1    False
2    False
3    False
4     True
Name: is_emergency, dtype: bool


In [12]:
''' Split dataframe into specific subject tables to build RDS '''

patients = df[['patient_id', 'age', 'gender', 'payer_id']]
payers = df[['payer_id', 'payer_name', 'payer_type']]
appointments = df[['appointment_id', 'patient_id', 'visit_date', 'visit_type', 'is_emergency', 'visit_reason', 'diagnosis', 'physician_id']]
doctors = df[['physician_id', 'department']]
billings = df[['appointment_id', 'admission_date', 'discharge_date', 'charge_amount_USD', 'claim_status', 'payment_amount_USD']]

billings_dataframes = ['patients', 'appointments', 'doctors', 'billings']

for table in billings_dataframes:
    print(f'\n{table} preview:')
    print(eval(table).head())


patients preview:
   patient_id  age  gender    payer_id
0      154669   43    Male  INS-MMIAPD
1      144776   56  Female  INS-M2HVS3
2      217503   49    Male  INS-GMOCQU
3      174851    8    Male    INS_NULL
4      193134   39  Female    INS_NULL

appointments preview:
  appointment_id  patient_id  visit_date  visit_type  is_emergency  \
0         A00000      154669  2024-11-05  Outpatient          True   
1         A00001      144776  2024-12-15  Outpatient         False   
2         A00002      217503  2024-01-02   Inpatient         False   
3         A00003      174851  2024-06-29  Outpatient         False   
4         A00004      193134  2024-11-21  Outpatient          True   

        visit_reason      diagnosis  physician_id  
0                NaN   Hypertension         30927  
1         Chest Pain   Hypertension         14816  
2  Surgery Follow-up      Back Pain         73819  
3              Fever         Asthma         81325  
4         Chest Pain  Heart Failure        

In [14]:
''' Save to separate csv files into cwd folder '''
working_dir = Path.cwd().parent/'data'/'split_data'
working_dir.mkdir(exist_ok=True)


for table in billings_dataframes:
    eval(table).to_csv(working_dir/f'{table}.csv', index=False)

print('\nSplit data saved to separate csv files under data/split_data.')   


Split data saved to separate csv files under data/split_data.
